ref: https://youtu.be/H9OmsD7F7p0

In [1]:
!pip install pythainlp
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.6 MB 19.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = '/content/drive/MyDrive/Final_project/clean_data/combine_company.csv'

In [4]:
import pandas as pd
import numpy as np
from pythainlp.corpus import thai_stopwords
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [101]:
df = pd.read_csv(data)
frozen_stopword = thai_stopwords()

In [6]:
# Clean stop word and special charater
import string
from pythainlp.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

english_stopword = stopwords.words('english')
thai_stopword = list(frozen_stopword)

def clean_tokenize(rec_text:str) -> list:
  clean_token = rec_text
  for pair in (('\n', ''), ('\t', ''), ('!', '')):
        clean_token =clean_token.replace(*pair)
  
  temp_tokenize = word_tokenize(clean_token, None, 'newmm', False)
  return temp_tokenize

def clean_stopword(word:list):
  temp_thai = []
  temp_eng = []
  for i in word:
    if i not in thai_stopword:
      temp_thai.append(i)
  
  for j in temp_thai:
    if j not in english_stopword:
      temp_eng.append(j)
    
  return temp_eng

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [102]:
# List company detail
"""
  ตัดรายละเอียดธุรกิจเป็นคำๆ แล้วใส่ในตัวแปร list_company_detail
"""

list_company_detail = []
for i in range(len(df)):
  detail = df.iloc[i]['รายละเอียดธุรกิจ'].lower()
  a = detail.translate(str.maketrans('','', string.punctuation))
  b = a.translate(str.maketrans('','', string.digits))
  temp_comp = clean_tokenize(b)
  stopword_cleaned = clean_stopword(temp_comp)
  list_company_detail.append(stopword_cleaned)

# หาค่า TF-IDF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
def identity_fun(text):
    return text

In [103]:
tfidf_vectorizer = TfidfVectorizer(
    stop_words='english',
    analyzer='word',
    tokenizer=identity_fun,
    preprocessor=identity_fun,
    token_pattern=None
)

X = tfidf_vectorizer.fit_transform(list_company_detail)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn(


# KMeans clustering

In [104]:
from sklearn.cluster import KMeans
k = 15
km = KMeans(n_clusters=k)
km.fit(X)
centroid = km.cluster_centers_
cluster = km.labels_
df['cluster'] = cluster

In [127]:
import random
temp_cluster = []
for i in range(len(df)):
  ind = random.randint(0, len(df)-1)
  temp = [df.iloc[ind]['ชื่อย่อสถานประกอบการ'], df.iloc[ind]['รายละเอียดธุรกิจ']]
  if df.iloc[ind]['cluster'] == 0 and len(temp_cluster) < 10 and temp not in temp_cluster:
    temp_cluster.append(temp)


for k in temp_cluster:
  print(k[0])
  print(k[1])
  print()

BPS
บริการ e-payment and card technologies (Central Clearing House (CCH) Solutions, Automated Fare Collection (AFC) / e-Ticket Solutions, Card and Retail Payment Solutions, Payment Terminals Solutions and Services)

OPTIMUS
บริการขายส่งคอมพิวเตอร์ อุปกรณ์ต่อพ่วงคอมพิวเตอร์และซอฟต์แวร์ (Wireless/Wired/Security Solutions, Data Analytics Solutions, Value Added Services, Collaborative Solutions)

CRM-C
บริการให้คำปรึกษาและการจัดหาเกี่ยวกับซอฟต์แวร์์ CRM และ Cloud Solutions

codemate
บริการออกแบบพัฒนา Web/Mobile Application, IoT Solutions

st
บริการระบบเทคโนโลยีสารสนเทศ ด้าน Travel solutions



In [168]:
group = []
name = []
for i in range(len(df)):
  if df.iloc[i]['cluster'] == 14:
    name.append(df.iloc[i]['ชื่อย่อสถานประกอบการ'])
    group.append(df.iloc[i]['รายละเอียดธุรกิจ'])

sample = pd.DataFrame({
    'name': name,
    'detail': group
})

cluster_list = {
    0: "IT Solution / Travel solutions",
    1: "Web Design & Development",
    2: 'Customer platform service POS / *',
    3: "IT solution",
    4: "Website development",
    5: "Biometric / Medical",
    6: "Digital agency / Digital marketing / SEO / Online marketing",
    7: "Online platform / Transport / Financial",
    8: "Network Infrastructure / Cloud service / Digital Consulting Services",
    9: "Software development *",
    10: "Chemical industry / Cosmetic manufacturer / Food / Industrial materials",
    11: "Enterprise Resource Planning / Accounting / Human resource management",
    12: "AR / VR / Automation system / Enterprise Solutions / Medical equipment / Construction / *",
    13: "Software Development / Desktop Applications",
    14: "Customer Relationship Management / Data analytics / AI / Big data / Chatbot"
}

sample

,name,detail
0,CC ChocoCRM,บริการ Platform CRM (customer relationship man...
1,VEKIN,บริการระบบบริหารจัดการพลังงานที่สามารถแสดงค่าก...
2,ZTRUS,บริการการแปลงข้อมูลจากเอกสารที่เป็นกระดาษ เข้า...
3,Ricult,บริการระบบจัดหาสินค้าการเกษตร (Digital Marketp...
4,Stelligent,บริการ Big Data / Business Intelligent / Artif...
...,...,...
63,MP Engineering System,บริการ นวัตกรรมที่รวบรวมข้อมูล Big data เครื่อ...
64,CORALINE,บริการที่ปรึกษา และสร้าง Solution โดยใช้ Big D...
65,R&DBI,บริการห้คำปรึกษาทางด้านการนำเสนอภาพข้อมูลทางธุ...
66,Aimagin,บริการผลิต ขาย อุปกรณ์อิเล็คทรอนิคส์ ระบบสมองก...


In [169]:
cluster_map = {
    0: "IT Solution / Travel solutions",
    1: "Web Design & Development",
    2: 'Customer platform service POS / *',
    3: "IT solution",
    4: "Website development",
    5: "Biometric / Medical",
    6: "Digital agency / Digital marketing / SEO / Online marketing",
    7: "Online platform / Transport / Financial",
    8: "Network Infrastructure / Cloud service / Digital Consulting Services",
    9: "Software development *",
    10: "Chemical industry / Cosmetic manufacturer / Food / Industrial materials",
    11: "Enterprise Resource Planning / Accounting / Human resource management",
    12: "AR / VR / Automation system / Enterprise Solutions / Medical equipment / Construction / *",
    13: "Software Development / Desktop Applications",
    14: "Customer Relationship Management / Data analytics / AI / Big data / Chatbot"
}

# apply mapping
df['cluster'] = df['cluster'].map(cluster_map)

In [174]:
import random
for i in range(5):
  ind_sample = random.randint(0, len(df))
  print(f'{df.iloc[ind_sample]["ชื่อย่อสถานประกอบการ"]}')
  print(df.iloc[ind_sample]['รายละเอียดธุรกิจ'])
  print(f'cluster label: {cluster[ind_sample]}')
  print(f"Category: {df.iloc[ind_sample]['cluster']}")
  print()

BIZNEXT
บริการแอพพลิเคชั่น สำหรับช่วยในการลดขั้นตอน และกระบวนการในการตรวจสอบประเมิน สินค้าและบริการเพื่อการควบคุมคุณภาพ
cluster label: 12
Category: AR / VR / Automation system / Enterprise Solutions / Medical equipment / Construction / *

UPPERCUZ CREATIVE
บริการลักษณะ Digital Agency ให้บริการด้านการตลาดออนไลน์ บน Facebook, Website, Instagram และYoutube (Online Marketing, Media Advertising, Website & Mobile App, Packaging Design, Branding & Consulting)
cluster label: 6
Category: Digital agency / Digital marketing / SEO / Online marketing

Raimaijon
บริการผลิตน้ำผลไม้และน้ำผัก น้ำอ้อยพาสเจอร์ไรส์ ผลิตภัณฑ์น้ำอ้อยธรรมชาติ 100% พาสเจอร์ไรซ์ ไม่แต่งสีแต่งกลิ่น และไม่ใส่วัตถุกันเสีย โดยคัดเลือกอ้อยพันธุ์เฉพาะอันดับหนึ่งของไทย คือ พันธุ์สุพรรณบุรี 50 ที่มีความหวาน กลิ่นหอม สีสวย จากไร่อ้อยกว่า 40 ครัวเรือน ครอบคลุม 3 จังหวัด สามารถยืดอายุการเก็บรักษาได้ยาวนานขึ้นกว่าเดิมถึง 6.7 เท่า
cluster label: 10
Category: Chemical industry / Cosmetic manufacturer / Food / Industrial materials

mirum
บริ

In [122]:
import collections
counter = collections.Counter(df['cluster'])
dict(sorted(counter.items()))

{0: 10,
 1: 79,
 2: 234,
 3: 65,
 4: 90,
 5: 28,
 6: 63,
 7: 122,
 8: 76,
 9: 24,
 10: 176,
 11: 66,
 12: 442,
 13: 100,
 14: 68}